In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import math

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.grid_search import GridSearchCV
from sklearn import svm
from sklearn.decomposition import RandomizedPCA
from sklearn.metrics import classification_report
from sklearn import preprocessing
import matplotlib.pyplot as plt
import scipy.optimize as opt
import glob
import os
import sys
import pickle
import time
import timeit

from sklearn.decomposition import RandomizedPCA

import theano
import theano.tensor as T
import lasagne

#np.random.seed(1337)  # for reproducibility
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Layer, Reshape, AutoEncoder, Merge
from keras.layers.convolutional import Convolution2D,MaxPooling2D,Convolution1D,MaxPooling1D,UpSampling2D, ZeroPadding2D
from keras.optimizers import SGD, Adam, RMSprop, Adadelta
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
import keras.utils.visualize_util as vutil
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.regularizers import l1, l2, l1l2, activity_l2, activity_l1
import keras.layers.containers as containers
from keras.layers.noise import GaussianNoise
from keras.regularizers import ActivityRegularizer
from keras import backend as K
import keras.utils.visualize_util as vutil
from keras.models import model_from_json

import seaborn as sns

%matplotlib inline

from IPython.display import SVG

import h5py

np.set_printoptions(suppress=True)  #Supress exponent of the number 2e-2 == 0.02

%matplotlib inline

WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: no CUDA-capable device is detected)
/usr/local/lib/python2.7/dist-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")
Using Theano backend.


In [2]:
#IS3

In [3]:
#Read the data

GenomicFeatures_IS3 = pd.read_csv(open('data/IS3/GenomicFeatures_IS3.txt','r'),sep='\t')
SNVCalls_IS3    = pd.read_csv(open('data/IS3/SNVCalls_IS3.txt','r'),sep='\t') 

/usr/lib/python2.7/dist-packages/pandas/io/parsers.py:1070: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [4]:
print GenomicFeatures_IS3.shape
print SNVCalls_IS3.shape

(22885, 16)
(22885, 70)


In [5]:
np.sum(GenomicFeatures_IS3.isnull())

CHROM                 0
POS                   0
END                   0
RefAllele             0
NonRefAllele          0
BaseQual              0
TumourCoverage        0
NormalCoverage        0
MapQual               0
ReadPosition          1
Trinucleotide         0
HomopolymerRate       0
GCcontent             0
GenomicLocation    1601
Distance              0
StrandBias            1
dtype: int64

In [8]:
GenomicFeatures_IS3['ReadPosition'].fillna(0, inplace=True)  #Replace na with 0
GenomicFeatures_IS3['StrandBias'].fillna(0, inplace=True)

#print np.sum(GenomicFeatures_IS3.isnull())

In [9]:
np.unique(GenomicFeatures_IS3['GenomicLocation'])

array(['UTR3', 'UTR5', 'downstream', 'exonic', 'exonic;splicing',
       'intergenic', 'intronic', 'nan', 'ncRNA_UTR3', 'ncRNA_exonic',
       'ncRNA_intronic', 'splicing', 'upstream', 'upstream;downstream'], 
      dtype='|S19')

In [10]:
GenomicFeatures_IS3.head()

,CHROM,POS,END,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,Distance,StrandBias
0,1,17537,17538,110,20,4254,148,132,37,54.0,GCC,2.707143,0.673267,ncRNA_intronic,37176,0.432432
1,1,27195,27196,26,6,1034,43,43,0,73.0,TCT,2.940741,0.420792,ncRNA_intronic,27518,0.255814
2,1,31158,31159,44,3,1484,67,50,0,63.0,TCC,2.683453,0.445545,intergenic,23555,0.641791
3,1,101685,101686,42,17,1234,65,49,13,19.0,TTT,4.142857,0.371287,intergenic,15566,0.276923
4,1,104234,104235,62,4,2137,72,66,0,38.5,TCA,2.459459,0.410891,intergenic,14382,0.555556


In [11]:
SNVCalls_IS3.head()

,CHROM,POS,END,X2405012,X2410591,X2420789,X2420793,X2425329,X2426321,X2426377,X2427156,X2427522,X2430887,X2430956,X2431183,X2431851,X2436976,X2445117,X2445210,X2445581,
0,1,17537,17538,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
1,1,27195,27196,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
2,1,31158,31159,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...
3,1,101685,101686,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
4,1,104234,104235,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...


In [12]:
np.unique(GenomicFeatures_IS3['Trinucleotide'])

array(['ACA', 'ACC', 'ACG', 'ACT', 'ATA', 'ATC', 'ATG', 'ATT', 'CCA',
       'CCC', 'CCG', 'CCT', 'CTA', 'CTC', 'CTG', 'CTT', 'GCA', 'GCC',
       'GCG', 'GCT', 'GTA', 'GTC', 'GTG', 'GTT', 'TCA', 'TCC', 'TCG',
       'TCT', 'TTA', 'TTC', 'TTG', 'TTT'], 
      dtype='|S3')

In [13]:
IS3 = pd.merge(GenomicFeatures_IS3, SNVCalls_IS3,left_on=['CHROM', 'POS'], right_on=['CHROM','POS'])


In [14]:
IS3.head() #182+15

,CHROM,POS,END_x,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,Distance,StrandBias,END_y,X2405012,X2410591,X2420789,
0,5,70741113,70741114,30,6,1173,39,31,46,58.0,GCT,2.828358,0.331683,intergenic,1304,0.641026,70741114,0,0,0,...
1,5,70830197,70830198,32,15,1367,58,46,60,47.5,CCT,2.313333,0.361386,intronic,147,0.568966,70830198,1,1,1,...
2,5,70912022,70912023,27,12,1271,47,60,60,40.0,ATC,2.514085,0.470297,intronic,1712,0.425532,70912023,1,1,1,...
3,5,70999477,70999478,28,0,305,39,46,39,19.0,TTT,8.016529,0.381188,intergenic,484,0.384615,70999478,0,0,0,...
4,5,71032072,71032073,15,0,484,38,40,29,79.0,TTC,4.663551,0.346535,intergenic,129,0.526316,71032073,0,0,0,...


In [15]:
IS3_X = IS3.iloc[:,3:16] #Exclude CHROM, POS
IS3_Y = IS3.iloc[:,17:]  #Exclude CHROM_y, POS_y, END

In [16]:
print IS3_X.shape
print IS3_Y.shape

(6501, 13)
(6501, 67)


In [17]:
IS3_X[:2]

,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,Distance,StrandBias
0,30,6,1173,39,31,46,58.0,GCT,2.828358,0.331683,intergenic,1304,0.641026
1,32,15,1367,58,46,60,47.5,CCT,2.313333,0.361386,intronic,147,0.568966


In [18]:
IS3_Y[:2]

,X2405012,X2410591,X2420789,X2420793,X2425329,X2426321,X2426377,X2427156,X2427522,X2430887,X2430956,X2431183,X2431851,X2436976,X2445117,X2445210,X2445581,X2446437,X2449297,X2449602,
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...


In [19]:
totalSample = IS3_Y.shape[0]
IS3_Y_Val = np.zeros((totalSample,1)) #Will be our label now

print totalSample
print IS3_Y_Val.shape

6501
(6501, 1)


In [20]:
IS3_Y.shape

(6501, 67)

In [21]:
IS3_Y.values[0].shape

(67,)

In [22]:
for i in range(totalSample):
    IS3_Y_Val[i] = np.argmax(np.bincount(IS3_Y.values[i]))

In [23]:
IS3_Y_Val_1 = pd.DataFrame(IS3_Y_Val)
IS3_Y_Val_1.columns = ['Label']

In [24]:
IS3_Y_Val_1[:3]

,Label
0,0
1,1
2,1


In [25]:
IS3_dataframe = pd.concat([IS3_X, IS3_Y_Val_1],axis=1)

In [26]:
IS3_dataframe.shape

(6501, 14)

In [27]:
IS3_dataframe[:2]

,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,Distance,StrandBias,Label
0,30,6,1173,39,31,46,58.0,GCT,2.828358,0.331683,intergenic,1304,0.641026,0
1,32,15,1367,58,46,60,47.5,CCT,2.313333,0.361386,intronic,147,0.568966,1


In [28]:
IS3_dataframe.to_csv('IS3.csv')

In [29]:
#Now we have variable importance ...lets do analysis

In [30]:
%load_ext rpy2.ipython

In [31]:
%%R
# Read in the data
IS3Frame = read.csv("IS3.csv")
str(IS3Frame)

'data.frame':	6501 obs. of  15 variables:
 $ X              : int  0 1 2 3 4 5 6 7 8 9 ...
 $ RefAllele      : int  30 32 27 28 15 20 30 28 22 36 ...
 $ NonRefAllele   : int  6 15 12 0 0 5 14 5 0 3 ...
 $ BaseQual       : int  1173 1367 1271 305 484 825 1195 1097 657 1291 ...
 $ TumourCoverage : int  39 58 47 39 38 28 50 37 27 47 ...
 $ NormalCoverage : int  31 46 60 46 40 31 53 41 32 37 ...
 $ MapQual        : num  46 60 60 39 29 60 60 20 29 60 ...
 $ ReadPosition   : num  58 47.5 40 19 79 57 53 35 70 42 ...
 $ Trinucleotide  : Factor w/ 32 levels "ACA","ACC","ACG",..: 20 12 6 32 30 4 30 27 24 25 ...
 $ HomopolymerRate: num  2.83 2.31 2.51 8.02 4.66 ...
 $ GCcontent      : num  0.332 0.361 0.47 0.381 0.347 ...
 $ GenomicLocation: Factor w/ 14 levels "","downstream",..: 5 6 6 5 5 5 5 6 6 6 ...
 $ Distance       : int  1304 147 1712 484 129 1058 1442 1466 0 256 ...
 $ StrandBias     : num  0.641 0.569 0.426 0.385 0.526 ...
 $ Label          : num  0 1 1 0 0 1 1 0 0 0 ...


In [33]:
%%R
#sum(is.na(CPCG0183Frame$StrandBias))
#sum(is.na(IS1Frame$ReadPosition)) 
sum(is.na(IS3Frame)) 

[1] 0


In [33]:
#%%R
#CPCG0235Frame$ReadPosition[is.na(CPCG0235Frame$ReadPosition)] = mean(CPCG0235Frame$ReadPosition, na.rm=TRUE)
#CPCG0235Frame$StrandBias[is.na(CPCG0235Frame$StrandBias)] = mean(CPCG0235Frame$StrandBias, na.rm=TRUE)

#print (sum(is.na(CPCG0235Frame$ReadPosition)) )
#print (sum(is.na(CPCG0235Frame$StrandBias)) )

In [34]:
%%R
sum(is.na(IS3Frame))  #No NA value now

[1] 0


In [35]:
%%R
write.csv(IS3Frame, file = "IS3_imputed.csv")  #Cleaned CSV without NA

In [36]:
%%R
library(h2o)
localH2O = h2o.init(ip = "localhost", port = 54321, startH2O = TRUE,min_mem_size = "3g")

/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: Loading required package: statmod

  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: 
----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit http://docs.h2o.ai

----------------------------------------------------------------------


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: ‘h2o’


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/usr/local/lib/python2.7/dist-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘

 Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         26 minutes 43 seconds 
    H2O cluster version:        3.9.1.3435 
    H2O cluster name:           H2O_started_from_R_rbhat_xqb196 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   26.58 GB 
    H2O cluster total cores:    56 
    H2O cluster allowed cores:  2 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    R Version:                  R version 3.3.0 beta (2016-03-30 r70404) 



In [37]:
%%R
## Import CSV as H2O:
csvPath = "/home/rbhat/Workspace/S3Lab_Projects/SMC-MetaChallange/IS3_imputed.csv"
IS3.hex          = h2o.importFile(path = csvPath, destination_frame = "IS3.hex") 
train                 = as.data.frame(IS3.hex)

str(train)

  |======================================================================| 100%
'data.frame':	6501 obs. of  16 variables:
 $ C1             : int  1 2 3 4 5 6 7 8 9 10 ...
 $ X              : int  0 1 2 3 4 5 6 7 8 9 ...
 $ RefAllele      : int  30 32 27 28 15 20 30 28 22 36 ...
 $ NonRefAllele   : int  6 15 12 0 0 5 14 5 0 3 ...
 $ BaseQual       : int  1173 1367 1271 305 484 825 1195 1097 657 1291 ...
 $ TumourCoverage : int  39 58 47 39 38 28 50 37 27 47 ...
 $ NormalCoverage : int  31 46 60 46 40 31 53 41 32 37 ...
 $ MapQual        : num  46 60 60 39 29 60 60 20 29 60 ...
 $ ReadPosition   : num  58 47.5 40 19 79 57 53 35 70 42 ...
 $ Trinucleotide  : Factor w/ 32 levels "ACA","ACC","ACG",..: 20 12 6 32 30 4 30 27 24 25 ...
 $ HomopolymerRate: num  2.83 2.31 2.51 8.02 4.66 ...
 $ GCcontent      : num  0.332 0.361 0.47 0.381 0.347 ...
 $ GenomicLocation: Factor w/ 13 levels "downstream","exonic",..: 4 5 5 4 4 4 4 5 5 5 ...
 $ Distance       : int  1304 147 1712 484 129 1058 1442 14

In [38]:
%%R

train$Trinucleotide   = as.factor(train$Trinucleotide)
train$GenomicLocation = as.factor(train$GenomicLocation)
train$Label           = as.factor(train$Label)

train_h2o             = as.h2o(train)

  |======================================================================| 100%


In [39]:
%%R
#Training a deep learning model:---------------------------------------------------------#
model <- h2o.deeplearning( x = 3:15, # column numbers for predictors
                           y = 16,            # column number for label
                           training_frame = train_h2o,
                           activation = "RectifierWithDropout", 
                           input_dropout_ratio = 0.2,
                           hidden_dropout_ratios = c(0.5,0.5),
                           balance_classes = TRUE, 
                           hidden = c(10,8),  ## two hidden layers
                           epochs = 50,
                           variable_importances = T)

  |======================================================================| 100%


In [40]:
%%R
var_importance = h2o.varimp(model)
#print(var_importance)
h2o.varimp(model)

Variable Importances: 
                     variable relative_importance scaled_importance percentage
1                NonRefAllele            1.000000          1.000000   0.078199
2                   RefAllele            0.578058          0.578058   0.045204
3                    BaseQual            0.568464          0.568464   0.044453
4              NormalCoverage            0.556932          0.556932   0.043552
5 GenomicLocation.missing(NA)            0.501616          0.501616   0.039226

---
                     variable relative_importance scaled_importance percentage
53          Trinucleotide.CCA            0.104141          0.104141   0.008144
54                  GCcontent            0.096585          0.096585   0.007553
55 GenomicLocation.intergenic            0.075224          0.075224   0.005882
56               ReadPosition            0.061978          0.061978   0.004847
57                 StrandBias            0.040941          0.040941   0.003202
58  Trinucleotide.missin

In [41]:
%%R
write.table(var_importance, file="IS3_variable_importance.csv")

In [42]:
var_importance = pd.read_csv(open('IS3_variable_importance.csv','r'),sep=' ')
#pd.read_csv('CPCG0100_variable_importance.csv')

In [43]:
var_importance[:5]

,variable,relative_importance,scaled_importance,percentage
1,NonRefAllele,1.000000,1.000000,0.078199
2,RefAllele,0.578058,0.578058,0.045204
3,BaseQual,0.568464,0.568464,0.044453
4,NormalCoverage,0.556932,0.556932,0.043552
5,GenomicLocation.missing(NA),0.501616,0.501616,0.039226


In [44]:
GenomicFeatures_IS3[:5]

,CHROM,POS,END,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,Distance,StrandBias
0,1,17537,17538,110,20,4254,148,132,37,54.0,GCC,2.707143,0.673267,ncRNA_intronic,37176,0.432432
1,1,27195,27196,26,6,1034,43,43,0,73.0,TCT,2.940741,0.420792,ncRNA_intronic,27518,0.255814
2,1,31158,31159,44,3,1484,67,50,0,63.0,TCC,2.683453,0.445545,intergenic,23555,0.641791
3,1,101685,101686,42,17,1234,65,49,13,19.0,TTT,4.142857,0.371287,intergenic,15566,0.276923
4,1,104234,104235,62,4,2137,72,66,0,38.5,TCA,2.459459,0.410891,intergenic,14382,0.555556


In [45]:
var_imp_dict = {}

for i in range(var_importance.shape[0]):
    var_imp_dict[var_importance.iloc[i,0]] = var_importance.iloc[i,1]

var_imp_dict

{'BaseQual': 0.56846415996551503,
 'Distance': 0.3871206343173979,
 'GCcontent': 0.096584580838680295,
 'GenomicLocation.UTR3': 0.18219177424907698,
 'GenomicLocation.UTR5': 0.17538018524646801,
 'GenomicLocation.downstream': 0.178618639707565,
 'GenomicLocation.exonic': 0.24255387485027302,
 'GenomicLocation.exonic;splicing': 0.22218076884746599,
 'GenomicLocation.intergenic': 0.075224064290523501,
 'GenomicLocation.intronic': 0.164586767554283,
 'GenomicLocation.missing(NA)': 0.50161641836166393,
 'GenomicLocation.ncRNA_UTR3': 0.14711047708988198,
 'GenomicLocation.ncRNA_exonic': 0.19320553541183497,
 'GenomicLocation.ncRNA_intronic': 0.13168583810329401,
 'GenomicLocation.splicing': 0.218622431159019,
 'GenomicLocation.upstream': 0.17438788712024697,
 'GenomicLocation.upstream;downstream': 0.22489254176616699,
 'HomopolymerRate': 0.173455104231834,
 'MapQual': 0.47048330307006797,
 'NonRefAllele': 1.0,
 'NormalCoverage': 0.55693233013153098,
 'ReadPosition': 0.061977840960025801,
 '

In [46]:
GenomicFeatures_IS3.columns

Index([u'CHROM', u'POS', u'END', u'RefAllele', u'NonRefAllele', u'BaseQual', u'TumourCoverage', u'NormalCoverage', u'MapQual', u'ReadPosition', u'Trinucleotide', u'HomopolymerRate', u'GCcontent', u'GenomicLocation', u'Distance', u'StrandBias'], dtype='object')

In [47]:
GenomicFeatures_IS3.dtypes

CHROM               object
POS                  int64
END                  int64
RefAllele            int64
NonRefAllele         int64
BaseQual             int64
TumourCoverage       int64
NormalCoverage       int64
MapQual            float64
ReadPosition       float64
Trinucleotide       object
HomopolymerRate    float64
GCcontent          float64
GenomicLocation     object
Distance             int64
StrandBias         float64
dtype: object

In [48]:
GenomicFeatures_IS3.iloc[0,13]

'ncRNA_intronic'

In [49]:
GenomicFeatures_IS3[:2]
#print GenomicFeatures_IS1.iloc[0,2]

,CHROM,POS,END,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,Distance,StrandBias
0,1,17537,17538,110,20,4254,148,132,37,54,GCC,2.707143,0.673267,ncRNA_intronic,37176,0.432432
1,1,27195,27196,26,6,1034,43,43,0,73,TCT,2.940741,0.420792,ncRNA_intronic,27518,0.255814


In [50]:
noOfSample = GenomicFeatures_IS3.shape[0]

IS3_Dict = {}

for i in range(noOfSample):
    
    pos = GenomicFeatures_IS3.iloc[i,1] #gets the pos value
    
    RefAllele      = GenomicFeatures_IS3.iloc[i,3]*var_imp_dict['RefAllele']
    NonRefAllele   = GenomicFeatures_IS3.iloc[i,4]*var_imp_dict['NonRefAllele']
    BaseQual       = GenomicFeatures_IS3.iloc[i,5]*var_imp_dict['BaseQual']
    TumourCoverage = GenomicFeatures_IS3.iloc[i,6]*var_imp_dict['TumourCoverage']
    NormalCoverage = GenomicFeatures_IS3.iloc[i,7]*var_imp_dict['NormalCoverage']
    MapQual        = GenomicFeatures_IS3.iloc[i,8]*var_imp_dict['MapQual']
    ReadPosition   = GenomicFeatures_IS3.iloc[i,9]*var_imp_dict['ReadPosition']
    
    Trinucleotide  = 0
    if isinstance(GenomicFeatures_IS3.iloc[i,10], str):
        Trinucleotide  = var_imp_dict['Trinucleotide.'+str(GenomicFeatures_IS3.iloc[i,10])]
    HomopolymerRate= GenomicFeatures_IS3.iloc[i,11]*var_imp_dict['HomopolymerRate']
    GCcontent      = GenomicFeatures_IS3.iloc[i,12]*var_imp_dict['GCcontent']
    
    GenomicLocation = 0
    if isinstance(GenomicFeatures_IS3.iloc[i,13], str):
        GenomicLocation= var_imp_dict['GenomicLocation.'+str(GenomicFeatures_IS3.iloc[i,13])]
    Distance    = GenomicFeatures_IS3.iloc[i,14]*var_imp_dict['Distance']
    StrandBias     = GenomicFeatures_IS3.iloc[i,15]*var_imp_dict['StrandBias']
    
    #print 'RefAllele=',RefAllele,' NonRefAllele=',NonRefAllele,' BaseQual=',BaseQual,' TumourCoverage=',TumourCoverage
    #print 'NormalCoverage=',NormalCoverage,' MapQual=',MapQual,' ReadPosition=',ReadPosition,' Trinucleotide=',Trinucleotide
    #print 'HomopolymerRate=',HomopolymerRate,' GCcontent=',GCcontent,' GenomicLocation=',GenomicLocation,' Distance=',Distance
    #print 'StrandBias=',StrandBias
    IS3_Dict[pos] = RefAllele+NonRefAllele+BaseQual+TumourCoverage+NormalCoverage+MapQual+ReadPosition+Trinucleotide+ \
                         HomopolymerRate+GCcontent+GenomicLocation+Distance+StrandBias

In [51]:
#Sorting dictionary
IS3_Dict1 = [(k,v) for v,k in sorted([(v,k) for k,v in IS3_Dict.items()],reverse=True)]

In [52]:
IS3_Dict_Frame = pd.DataFrame(IS3_Dict1, columns=['POS', 'Value'])

In [53]:
IS3_Dict_Frame.to_csv("IS3_Dict_Frame.csv")

In [54]:
IS3_Dict_Frame[:10]

,POS,Value
0,27791928,293455.596265
1,27791988,293446.207585
2,9723294,76856.085835
3,92324518,72316.352239
4,46587743,62913.635602
5,45874406,56438.413848
6,46370116,52613.234888
7,41084701,52079.755376
8,86732778,51757.153606
9,86747067,47623.014216


In [58]:
GenomicFeatures_IS3_1 = pd.concat([IS3.iloc[:,0:16], IS3_Y_Val_1],axis=1)

In [59]:
rows = IS3_Dict_Frame['POS'][:20].values
print rows
GenomicFeatures_IS3_Top20 = GenomicFeatures_IS3_1.loc[GenomicFeatures_IS3_1['POS'].isin(rows)]

[ 27791928  27791988   9723294  92324518  46587743  45874406  46370116
  41084701  86732778  86747067 110629414  42723515  42724344  42724350
  42733563  39346116  86740316  86754887  41120841  41122134]


In [61]:
GenomicFeatures_IS3_Top20[:10]

,CHROM,POS,END_x,RefAllele,NonRefAllele,BaseQual,TumourCoverage,NormalCoverage,MapQual,ReadPosition,Trinucleotide,HomopolymerRate,GCcontent,GenomicLocation,Distance,StrandBias,Label
3878,8,86732778,86732779,466,1,15852,573,506,0,43.0,CCT,2.246667,0.485149,intergenic,108399,0.582897,0
3879,8,86740316,86740317,56,9,2206,83,79,0,74.0,CTT,2.437500,0.485149,intergenic,100861,0.144578,0
3880,8,86747067,86747068,534,1,18026,668,655,0,69.0,GCC,2.500000,0.599010,intergenic,94110,0.413174,0
3881,8,86754887,86754888,291,16,10232,382,363,0,57.0,TCC,2.500000,0.440594,NaN,86290,0.628272,0
4544,9,39346116,39346117,15,4,576,19,26,0,76.0,ACG,3.006897,0.500000,intergenic,104047,0.263158,0
4571,9,41084701,41084702,10,10,562,23,16,0,66.0,TTG,2.333333,0.613861,intergenic,133606,0.521739,0
4572,9,41120841,41120842,9,3,300,13,12,15,25.0,TCT,5.496124,0.415842,intergenic,97466,0.538462,0
4573,9,41122134,41122135,14,4,555,19,17,0,45.0,ATG,2.489510,0.554455,intergenic,96173,0.368421,0
4597,9,42723515,42723516,26,2,860,38,38,0,72.5,ACA,2.423611,0.485149,intergenic,116909,0.947368,0
4598,9,42724344,42724345,10,1,351,24,30,0,37.0,TTG,2.424658,0.306931,intergenic,116720,0.250000,0


In [57]:
GenomicFeatures_IS3_Top20.to_csv('GenomicFeatures_IS3_Top20.csv')